# Enfermedad Dengue en la República Argentina - Aplicación de Machine Learning para su estudio y predicción de casos 
## Modelos 
#### Elaborado por: Fresca, Lorenzo - Mojico, Ailín- Rosselló, Matías
#### Ingeniería Industrial UTN FRBA

In [2]:
#Carga de archivos en Colab
from google.colab import files
upload=files.upload()

Saving Casos_Dengue_Procesados.csv to Casos_Dengue_Procesados.csv


In [3]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm 
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures,StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression , Ridge
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score
from sklearn import neighbors


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
ls

Casos_Dengue_Procesados.csv  sample_data/


In [6]:
#Leemos el dataframe ya pre-procesado
dengue=pd.read_csv("/content/Casos_Dengue_Procesados.csv", encoding = "latin")
dengue.dropna(how ='any', inplace = True)
dengue

,Provincia,Mes,Anio,Temperatura,Precipitacion,Humedad,Vel_vientos,cantidad_casos
0,BUENOS AIRES,1,2018,23.818903,78.826000,59.146452,14.287097,0
1,BUENOS AIRES,1,2019,22.901084,131.553399,69.955962,14.790099,0
2,BUENOS AIRES,1,2020,22.715584,79.471401,62.084416,15.177807,103
3,BUENOS AIRES,2,2018,23.143357,49.372000,62.202143,13.391429,9
4,BUENOS AIRES,2,2019,21.580357,60.700000,66.137143,12.844286,7
...,...,...,...,...,...,...,...,...
745,USHUAIA,10,2019,6.205645,47.900000,66.177419,28.225806,0
746,USHUAIA,11,2018,8.048333,23.500000,61.933333,30.483333,0
747,USHUAIA,11,2019,7.395833,50.100000,71.591667,22.641667,0
748,USHUAIA,12,2018,9.560484,24.200000,63.362903,25.620968,0


In [7]:
#Meses a analizar para predicción de datos
cant_mes= 6
cant_mes= cant_mes + 1
x=range(cant_mes)
list(x)
cant_meses=list(range(cant_mes))
dengue = dengue[dengue.Mes.isin(cant_meses)]
dengue

,Provincia,Mes,Anio,Temperatura,Precipitacion,Humedad,Vel_vientos,cantidad_casos
0,BUENOS AIRES,1,2018,23.818903,78.826000,59.146452,14.287097,0
1,BUENOS AIRES,1,2019,22.901084,131.553399,69.955962,14.790099,0
2,BUENOS AIRES,1,2020,22.715584,79.471401,62.084416,15.177807,103
3,BUENOS AIRES,2,2018,23.143357,49.372000,62.202143,13.391429,9
4,BUENOS AIRES,2,2019,21.580357,60.700000,66.137143,12.844286,7
...,...,...,...,...,...,...,...,...
733,USHUAIA,5,2019,4.674194,26.500000,73.217742,19.612903,0
734,USHUAIA,5,2020,5.681301,25.000000,75.008130,17.276423,0
735,USHUAIA,6,2018,2.753846,72.800000,74.648352,17.703297,0
736,USHUAIA,6,2019,2.339496,38.900000,80.546218,14.134454,0


In [8]:
#Creamos Dummies sobre las provincias
dengue_dummies=pd.get_dummies(dengue["Provincia"])


In [9]:
#Unimos ambas tablas y eliminamos la columna Provincia
dengue = pd.concat([dengue, dengue_dummies], axis=1)
dengue=dengue.drop(columns="Provincia")

In [10]:
#Acomodamos la columna target -cantidad de casos- como última columna del dataset
dengue_target=dengue.loc[:,"cantidad_casos"]
dengue=dengue.drop(columns="cantidad_casos")
dengue = pd.concat([dengue, dengue_target], axis=1)
dengue

,Mes,Anio,Temperatura,Precipitacion,Humedad,Vel_vientos,BUENOS AIRES,CABA,CATAMARCA,CHACO,CHUBUT,CORRIENTES,CÃRDOBA,ENTRE RÃOS,FORMOSA,JUJUY,LA PAMPA,LA RIOJA,MENDOZA,MISIONES,NEUQUÃN,RÃO NEGRO,SALTA,SAN JUAN,SAN LUIS,SANTA CRUZ,SANTA FE,SANTIAGO DEL ESTERO,TIERRA DEL FUEGO,TUCUMÃN,USHUAIA,cantidad_casos
0,1,2018,23.818903,78.826000,59.146452,14.287097,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2019,22.901084,131.553399,69.955962,14.790099,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2020,22.715584,79.471401,62.084416,15.177807,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103
3,2,2018,23.143357,49.372000,62.202143,13.391429,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
4,2,2019,21.580357,60.700000,66.137143,12.844286,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,5,2019,4.674194,26.500000,73.217742,19.612903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
734,5,2020,5.681301,25.000000,75.008130,17.276423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
735,6,2018,2.753846,72.800000,74.648352,17.703297,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
736,6,2019,2.339496,38.900000,80.546218,14.134454,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [11]:
#Guardamos las variables donde esté el target y la dimensión de las features
rows_features= np.shape(dengue)[1] - 1
rows_target=np.shape(dengue)[1]
dengue

,Mes,Anio,Temperatura,Precipitacion,Humedad,Vel_vientos,BUENOS AIRES,CABA,CATAMARCA,CHACO,CHUBUT,CORRIENTES,CÃRDOBA,ENTRE RÃOS,FORMOSA,JUJUY,LA PAMPA,LA RIOJA,MENDOZA,MISIONES,NEUQUÃN,RÃO NEGRO,SALTA,SAN JUAN,SAN LUIS,SANTA CRUZ,SANTA FE,SANTIAGO DEL ESTERO,TIERRA DEL FUEGO,TUCUMÃN,USHUAIA,cantidad_casos
0,1,2018,23.818903,78.826000,59.146452,14.287097,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2019,22.901084,131.553399,69.955962,14.790099,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2020,22.715584,79.471401,62.084416,15.177807,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,103
3,2,2018,23.143357,49.372000,62.202143,13.391429,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9
4,2,2019,21.580357,60.700000,66.137143,12.844286,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,5,2019,4.674194,26.500000,73.217742,19.612903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
734,5,2020,5.681301,25.000000,75.008130,17.276423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
735,6,2018,2.753846,72.800000,74.648352,17.703297,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
736,6,2019,2.339496,38.900000,80.546218,14.134454,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [12]:
#Dividimos las features en X y el target en Y
#Features
X= dengue.iloc[:,0:rows_features]  
#Target
Y= dengue.iloc[:,rows_target-1]

In [13]:
X

,Mes,Anio,Temperatura,Precipitacion,Humedad,Vel_vientos,BUENOS AIRES,CABA,CATAMARCA,CHACO,CHUBUT,CORRIENTES,CÃRDOBA,ENTRE RÃOS,FORMOSA,JUJUY,LA PAMPA,LA RIOJA,MENDOZA,MISIONES,NEUQUÃN,RÃO NEGRO,SALTA,SAN JUAN,SAN LUIS,SANTA CRUZ,SANTA FE,SANTIAGO DEL ESTERO,TIERRA DEL FUEGO,TUCUMÃN,USHUAIA
0,1,2018,23.818903,78.826000,59.146452,14.287097,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2019,22.901084,131.553399,69.955962,14.790099,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2020,22.715584,79.471401,62.084416,15.177807,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2,2018,23.143357,49.372000,62.202143,13.391429,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2,2019,21.580357,60.700000,66.137143,12.844286,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,5,2019,4.674194,26.500000,73.217742,19.612903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
734,5,2020,5.681301,25.000000,75.008130,17.276423,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
735,6,2018,2.753846,72.800000,74.648352,17.703297,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
736,6,2019,2.339496,38.900000,80.546218,14.134454,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [14]:
# Separamos train y test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=4)
print(x_train.shape)
print(y_train.shape)


(270, 31)
(270,)


In [15]:
x_train

,Mes,Anio,Temperatura,Precipitacion,Humedad,Vel_vientos,BUENOS AIRES,CABA,CATAMARCA,CHACO,CHUBUT,CORRIENTES,CÃRDOBA,ENTRE RÃOS,FORMOSA,JUJUY,LA PAMPA,LA RIOJA,MENDOZA,MISIONES,NEUQUÃN,RÃO NEGRO,SALTA,SAN JUAN,SAN LUIS,SANTA CRUZ,SANTA FE,SANTIAGO DEL ESTERO,TIERRA DEL FUEGO,TUCUMÃN,USHUAIA
523,5,2019,12.346341,4.524390,62.190244,7.839024,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
182,1,2020,24.343203,90.471429,65.784562,11.926267,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
221,4,2020,18.384242,113.727273,71.315152,9.963636,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,6,2019,11.143667,102.586000,81.225333,13.998000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
579,4,2018,8.765000,59.050000,62.470833,23.166667,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,3,2019,22.280645,77.800000,77.774194,8.088710,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
317,6,2020,8.490583,12.401794,74.443946,13.860987,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
727,3,2019,8.816935,44.400000,73.572581,18.338710,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
282,5,2018,12.462097,12.400000,62.399194,5.326613,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
#Escalamos los datos 
scaler = preprocessing.StandardScaler().fit(x_train)
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [17]:
x_train_scaled = scaler.transform(x_train) 
x_test_scaled= scaler.transform(x_test) 

In [18]:
#Creamos una matriz donde guardamos los resultados de cada modelo
resultados = pd.DataFrame(columns=['Modelo','R2','RMSE','MAE'])

In [19]:
#Definimos el estimador para una regresión lineal
est = LinearRegression()
#Definimos el hyper-parametro de regresión lineal, que nos dice si arrancamos desde el origen o no
parameters = {'fit_intercept' : [False, True]}
#Definimos la cantidad de folds para validar
n_folds = 5
#Definimos el objeto grid search con los parámetros anteriores
gs = GridSearchCV(est, param_grid=parameters, cv=n_folds, refit=True, scoring="neg_mean_squared_error")

In [20]:
#Entrenamos nuestro modelo
gs.fit(x_train_scaled, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'fit_intercept': [False, True]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [21]:
# Mostramos el mejor estimador, sus hiperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False) 

{'fit_intercept': True} 

-368439.4540846815 



In [22]:
# Usamos x_test para predecir con el mejor estimador
linear_prediction = gs.best_estimator_.predict(x_test_scaled)
#Calculamos las metricas R2, RMSE y MAE
linear_r2 = r2_score(y_true=y_test, y_pred=linear_prediction)
linear_rmse = mean_squared_error(y_true=y_test, y_pred=linear_prediction)
linear_mae = mean_absolute_error(y_true=y_test, y_pred=linear_prediction)

In [23]:
#Vamos a plotear Predictions vs True values
print(f'R2 score: {linear_r2:.6f}')
print(f'MAE: {linear_mae:.6f}')
print(f'RMSE: {linear_rmse:.6f}')

R2 score: 0.056678
MAE: 234.022931
RMSE: 137957.510123


In [24]:
#Guardamos los resultados
resultados = resultados.append({'Modelo':'Linear',
                                'R2':linear_r2,
                                'RMSE':linear_rmse,
                                'MAE':linear_mae},ignore_index=True)
resultados

,Modelo,R2,RMSE,MAE
0,Linear,0.056678,137957.510123,234.022931


In [25]:
# Ahora cambiamos el estimador, usaremos Ridge Regression para regularizar
est = Ridge()
# Lista del parametro lambda 
lambdas = [0.001,0.005,0.01,0.02,0.05,0.1,0.2,0.3,0.4,0.5,1]
parameters = {'alpha': lambdas}
# Definimos nuevamente el objeto GS con los parametros previamente definidos
gs = GridSearchCV(est, param_grid=parameters,refit=True, cv=n_folds, scoring="neg_mean_squared_error")

In [26]:
# Entrenamos
gs.fit(x_train_scaled, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [0.001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2,
                                   0.3, 0.4, 0.5, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [27]:
# Mostramos el mejor estimador, sus hyperparametros seleccionados y su resultado
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001) 

{'alpha': 1} 

-366873.06936503167 



In [28]:
#Predicciones + Metricas
ridge_prediction = gs.best_estimator_.predict(x_test_scaled)
ridge_r2 = r2_score(y_true=y_test, y_pred=ridge_prediction)
ridge_rmse = mean_squared_error(y_true=y_test, y_pred=ridge_prediction)
ridge_mae = mean_absolute_error(y_true=y_test, y_pred=ridge_prediction)

In [29]:
print(f'R2 score: {ridge_r2:.6f}')
print(f'RMSE: {ridge_rmse:.6f}')
print(f'MAE: {ridge_mae:.6f}')

R2 score: 0.060763
RMSE: 137360.102330
MAE: 232.822357


In [30]:
resultados = resultados.append({'Modelo':'Ridge Regression',
                                'R2':ridge_r2,
                                'RMSE':ridge_rmse,
                                'MAE':ridge_mae},ignore_index=True)
resultados

,Modelo,R2,RMSE,MAE
0,Linear,0.056678,137957.510123,234.022931
1,Ridge Regression,0.060763,137360.102330,232.822357


In [31]:
#Probamos con otro modelo de regularizción
from sklearn.linear_model import Lasso
lasso = Lasso()
lasso.fit(x_train_scaled,y_train)
train_score=lasso.score(x_train_scaled,y_train)
test_score=lasso.score(x_test_scaled,y_test)
y_pred_lasso = lasso.predict(x_test_scaled)
lasso_rmse = mean_squared_error(y_true=y_test, y_pred=y_pred_lasso)
lasso_mae = mean_absolute_error(y_true=y_test, y_pred=y_pred_lasso)
test_score


0.06275072092548561

In [32]:
resultados = resultados.append({'Modelo':'Lasso',
                                'R2':test_score,
                                'RMSE':lasso_rmse,
                                'MAE':lasso_mae},ignore_index=True)
resultados

,Modelo,R2,RMSE,MAE
0,Linear,0.056678,137957.510123,234.022931
1,Ridge Regression,0.060763,137360.102330,232.822357
2,Lasso,0.062751,137069.401930,231.725174


In [33]:
# Ahora cambiamos el estimador, usaremos SVR (Super Vector Regression)
est = SVR(max_iter=25000)
# Lista del parametros, con Kernels
parameters = {'kernel':('linear', 'rbf'),
    'C' : [1000,1500,2000,3000,5000],
              'epsilon' : [0.001, 0.01, 0.1,1, 10,100],
             'gamma':[0.001, 0.01, 0.1,1, 10,100]}
# Definimos el objeto GS con los parametros definidos
gs = GridSearchCV(est, param_grid=parameters,refit=True,
                  cv=n_folds, scoring="neg_mean_squared_error",
                  verbose=3, n_jobs=3)



In [34]:
gs.fit(x_train_scaled, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:    3.4s
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed:    8.8s
[Parallel(n_jobs=3)]: Done 585 tasks      | elapsed:   27.6s
[Parallel(n_jobs=3)]: Done 1481 tasks      | elapsed:  1.1min
[Parallel(n_jobs=3)]: Done 1795 out of 1800 | elapsed:  1.4min remaining:    0.2s
[Parallel(n_jobs=3)]: Done 1800 out of 1800 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=25000, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=3,
             param_grid={'C': [1000, 1500, 2000, 3000, 5000],
                         'epsilon': [0.001, 0.01, 0.1, 1, 10, 100],
                         'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=3)

In [35]:
print(gs.best_estimator_, "\n")
print(gs.best_params_, "\n")
print(gs.best_score_, "\n")

SVR(C=1500, cache_size=200, coef0=0.0, degree=3, epsilon=100, gamma=0.1,
    kernel='rbf', max_iter=25000, shrinking=True, tol=0.001, verbose=False) 

{'C': 1500, 'epsilon': 100, 'gamma': 0.1, 'kernel': 'rbf'} 

-347490.5495057312 



In [36]:
# Prediccion + Metricas
svr_prediction = gs.best_estimator_.predict(x_test_scaled)
svr_r2 = r2_score(y_true=y_test, y_pred=svr_prediction)
svr_rmse = mean_squared_error(y_true=y_test, y_pred=svr_prediction)
svr_mae = mean_absolute_error(y_true=y_test, y_pred=svr_prediction)

In [37]:
print(f'R2 score: {svr_r2:.6f}')
print(f'MAE: {svr_mae:.6f}')
print(f'RMSE: {svr_rmse:.6f}')

R2 score: 0.317942
MAE: 164.337727
RMSE: 99748.575227


In [38]:
resultados = resultados.append({'Modelo':'SVR',
                                'R2':svr_r2,
                                'RMSE':svr_rmse,
                                'MAE':svr_mae},ignore_index=True)
resultados

,Modelo,R2,RMSE,MAE
0,Linear,0.056678,137957.510123,234.022931
1,Ridge Regression,0.060763,137360.102330,232.822357
2,Lasso,0.062751,137069.401930,231.725174
3,SVR,0.317942,99748.575227,164.337727


In [39]:
#Probamos KNN Regressor testeando con diferentes K vecinos con un iterador
knn_r_acc = []
for i in range(1,17,1):
    knn = KNeighborsRegressor(n_neighbors=i)
    knn.fit(x_train_scaled,y_train)
    test_score = knn.score(x_test_scaled,y_test)
    train_score = knn.score(x_train_scaled,y_train)
    knn_r_acc.append((i, test_score ,train_score))
prueba_neighbors = pd.DataFrame(knn_r_acc, columns=['K','Test Score','Train Score'])
prueba_neighbors.sort_values('Test Score', ascending=False, inplace=True)
prueba_neighbors=prueba_neighbors.reset_index()
prueba_neighbors

,index,K,Test Score,Train Score
0,1,2,0.320381,0.668642
1,0,1,0.270510,1.000000
2,3,4,0.258963,0.393352
3,2,3,0.246648,0.488403
4,15,16,0.133013,0.137630
5,4,5,0.123317,0.346704
6,12,13,0.037720,0.104967
7,14,15,0.023474,0.139656
8,5,6,0.017082,0.273500
9,13,14,-0.006563,0.123386


In [40]:
k_elegido=prueba_neighbors["K"][0]
k_elegido

2

In [41]:
knn = KNeighborsRegressor(n_neighbors=k_elegido)
knn.fit(x_train_scaled,y_train)
test_score = knn.score(x_test_scaled,y_test)
train_score = knn.score(x_train_scaled,y_train)
predict_knn= knn.predict(x_test_scaled)
knn_rmse = mean_squared_error(y_true=y_test, y_pred=predict_knn)
knn_mae = mean_absolute_error(y_true=y_test, y_pred=predict_knn)

In [42]:
#Elegimos el mejor resultado obtenido
resultados = resultados.append({'Modelo':f'KNN (K={k_elegido})',
                                'R2':prueba_neighbors["Test Score"].max(),
                                'RMSE':knn_rmse,
                                'MAE':knn_mae},ignore_index=True)
resultados

,Modelo,R2,RMSE,MAE
0,Linear,0.056678,137957.510123,234.022931
1,Ridge Regression,0.060763,137360.102330,232.822357
2,Lasso,0.062751,137069.401930,231.725174
3,SVR,0.317942,99748.575227,164.337727
4,KNN (K=2),0.320381,99391.862500,95.441667


In [43]:
# Probamos Polynomial Regression 
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=4)
X_train_poly = poly_features.fit_transform(x_train_scaled)


In [44]:
pol_model = LinearRegression()
#Entramos al modelo de entrenamiento
pol_model.fit(X_train_poly, y_train)
y_train_pred = pol_model.predict(X_train_poly)

In [45]:
#Probamos al modelo de testeo
y_test_predict = pol_model.predict(poly_features.fit_transform(x_test_scaled))

In [46]:
#Evaluamos el modelo con los datos de entrenamiento
rmse= np.sqrt(mean_squared_error(y_train,y_train_pred))
pol_reg_r2 = r2_score(y_train,y_train_pred)
print(f'El error cuadrático medio de entremiento es {rmse}')
print(f'El Score es de entrenamiento es {pol_reg_r2}')

El error cuadrático medio de entremiento es 6.4556930628232944e-12
El Score es de entrenamiento es 1.0


In [47]:
#Evaluamos al modelo con los datos de testeo
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_predict))
r2_test = r2_score(y_test, y_test_predict)
pol_mae= mean_squared_error(y_true=y_test, y_pred=y_test_predict)
print(f'El Score es de testeo es {r2_test}')

El Score es de testeo es -3.2461941000403627


In [48]:
resultados = resultados.append({'Modelo':'Polynomial Regression',
                                'R2':r2_test,
                                'RMSE':rmse_test,
                                'MAE':pol_mae},ignore_index=True)
resultados

,Modelo,R2,RMSE,MAE
0,Linear,0.056678,137957.510123,234.022931
1,Ridge Regression,0.060763,137360.102330,232.822357
2,Lasso,0.062751,137069.401930,231.725174
3,SVR,0.317942,99748.575227,164.337727
4,KNN (K=2),0.320381,99391.862500,95.441667
5,Polynomial Regression,-3.246194,788.029767,620990.913268


In [49]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(x_train_scaled, y_train)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [50]:
predictions = rf.predict(x_test_scaled)
r2_forest = r2_score(y_test, predictions)
r2_forest
rmse_forest= mean_squared_error(y_true=y_test, y_pred=predictions)
mae_forest= mean_absolute_error(y_true=y_test, y_pred=predictions)

In [51]:
resultados = resultados.append({'Modelo':'Random Forest Regressor',
                                'R2':r2_forest,
                                'RMSE':rmse_forest,
                                'MAE':mae_forest},ignore_index=True)

In [52]:
resultados = resultados.sort_values('R2', ascending=False)
resultados

,Modelo,R2,RMSE,MAE
4,KNN (K=2),0.320381,99391.862500,95.441667
3,SVR,0.317942,99748.575227,164.337727
6,Random Forest Regressor,0.241314,110955.225157,124.674517
2,Lasso,0.062751,137069.401930,231.725174
1,Ridge Regression,0.060763,137360.102330,232.822357
0,Linear,0.056678,137957.510123,234.022931
5,Polynomial Regression,-3.246194,788.029767,620990.913268


In [53]:
resultados=resultados.reset_index()

In [54]:
print(f'El mejor estimador de regresión es: {resultados["Modelo"][0]} con un R2 de {round(resultados["R2"][0],4)}')

El mejor estimador de regresión es: KNN (K=2) con un R2 de 0.3204
